In [19]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd #DataFrame
import time  #for pausing loop to prevent excessive API requests
import sklearn #for normalizing data before visualizing

In [20]:
client_id = '###'
client_secret = '###'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [21]:
def getTrackIDs(user, playlist_id): #function for retrieving track IDs for each song in playlist
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [22]:
ids = getTrackIDs('1271437', '7Dht8VqWslmRqFAoipFd7W')

In [23]:
def gettrackfeatures(id):
    metadata = sp.track(id) #track information (e.g. artist, album, etc.)
    audiofeatures = sp.audio_features(id) #audio information (e.g. energy, liveness)
    
    name = metadata['name']
    album = metadata['album']['name']
    duration = metadata['duration_ms']
    popularity = metadata['popularity']
    
    acousticness = audiofeatures[0]['acousticness']
    danceability = audiofeatures[0]['danceability']
    energy = audiofeatures[0]['energy']
    instrumentalness = audiofeatures[0]['instrumentalness']
    liveness = audiofeatures[0]['liveness']
    loudness = audiofeatures[0]['loudness']
    speechiness = audiofeatures[0]['speechiness']
    tempo = audiofeatures[0]['tempo']
    
    track = [name, album, duration, popularity, acousticness, danceability, energy,
             instrumentalness, liveness, loudness, speechiness, tempo]
    return track

In [24]:
trackdata = [] #blank list
for i in range(len(ids)):
    time.sleep(.5) #delay to prevent too many API requests
    track = gettrackfeatures(ids[i]) #iterate through each ID
    trackdata.append(track) #add results to list

In [25]:
df = pd.DataFrame(trackdata, columns = ['Track', 'Album', 'Duration', 'Popularity', 'Acousticness',
                                    'Danceability', 'Energy', 'Instrumentalness', 'Liveness', 'Loudness', 'Speechiness',
                                    'Tempo'])

In [26]:
dfnotnormalized = df[['Track', 'Album']] #isolating non-quant metrics
dfnormalized = df[['Duration', 'Popularity', 'Acousticness',
                                    'Danceability', 'Energy', 'Instrumentalness', 'Liveness', 'Loudness', 'Speechiness',
                                    'Tempo']]

In [27]:
from sklearn.preprocessing  import MinMaxScaler #scaling the data for range 0 to 1
X = dfnormalized.copy()
scaler = MinMaxScaler()

scaler.fit(X)
X = pd.DataFrame(scaler.transform(X))
X.columns = dfnormalized.columns.values 

X.head()

,Duration,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo
0,0.528703,0.666667,0.554511,0.959732,0.612233,0.001209,0.357724,0.795680,0.547826,0.578270
1,0.545535,0.589744,0.783386,0.552573,0.341027,0.001046,0.376694,0.682745,0.242609,0.904121
2,0.579063,0.564103,0.149893,0.851230,0.609925,0.002770,0.085095,0.837924,0.233043,0.567117
3,0.448554,0.769231,0.632165,0.803132,0.592614,0.000708,0.165854,0.734908,0.250435,0.654340
4,0.559720,0.717949,0.390007,0.610738,0.678015,0.000011,0.266396,0.803283,0.221739,0.436374


In [28]:
concatdf = pd.concat([dfnotnormalized, X], axis=1) #concatenate non-quant metrics and normalized audio features
concatdf.head()

,Track,Album,Duration,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo
0,Still Feel Like Your Man,The Search for Everything,0.528703,0.666667,0.554511,0.959732,0.612233,0.001209,0.357724,0.795680,0.547826,0.578270
1,Emoji of a Wave,The Search for Everything,0.545535,0.589744,0.783386,0.552573,0.341027,0.001046,0.376694,0.682745,0.242609,0.904121
2,Helpless,The Search for Everything,0.579063,0.564103,0.149893,0.851230,0.609925,0.002770,0.085095,0.837924,0.233043,0.567117
3,Love on the Weekend,The Search for Everything,0.448554,0.769231,0.632165,0.803132,0.592614,0.000708,0.165854,0.734908,0.250435,0.654340
4,In the Blood,The Search for Everything,0.559720,0.717949,0.390007,0.610738,0.678015,0.000011,0.266396,0.803283,0.221739,0.436374


In [29]:
jmfinal = pd.melt(concatdf, id_vars=['Track', 'Album'], var_name = 'Metric', value_name = "Metric Value") 

#pivot DataFrame using pd.melt method from wide to long

jmfinal.head()

,Track,Album,Metric,Metric Value
0,Still Feel Like Your Man,The Search for Everything,Duration,0.528703
1,Emoji of a Wave,The Search for Everything,Duration,0.545535
2,Helpless,The Search for Everything,Duration,0.579063
3,Love on the Weekend,The Search for Everything,Duration,0.448554
4,In the Blood,The Search for Everything,Duration,0.559720


In [30]:
jmfinal.to_csv('jm.csv', index = False)